<a href="https://colab.research.google.com/github/1vanl0pez/DataScienceSantiagoIvan/blob/main/DSPRO1_HRAnalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries for data management and math operations
import pandas as pd
import numpy as np
# Import the random forest tool for regression models
from sklearn.ensemble import RandomForestRegressor
# Import the tool for K fold crros model validation
from sklearn.model_selection import KFold
# Import the tool to compute MSE
from sklearn.metrics import mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/general_data.xlsx'

Mounted at /content/drive


In [3]:
# Read the dataframe
df = pd.read_excel(file_path)
# Display the headers and the first values to confirm a correct read
df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,Female,1,Healthcare Representative,...,1.0,6,1,0,0,3.0,4.0,2.0,3,3
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,Female,1,Research Scientist,...,6.0,3,5,1,4,3.0,2.0,4.0,2,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,Male,4,Sales Executive,...,5.0,2,5,0,3,2.0,2.0,1.0,3,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,Male,3,Human Resources,...,13.0,5,8,7,5,4.0,4.0,3.0,2,3
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,Male,1,Sales Executive,...,9.0,2,6,0,4,4.0,1.0,3.0,3,3


In [4]:
# Compute main statistics
df.describe()

,Age,DistanceFromHome,Education,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
count,4410.000000,4410.000000,4410.000000,4410.000000,4410.000000,4391.000000,4410.000000,4410.000000,4401.000000,4410.000000,4410.000000,4410.000000,4410.000000,4385.000000,4390.000000,4372.000000,4410.000000,4410.000000
mean,36.923810,9.192517,2.912925,2.063946,65029.312925,2.694830,15.209524,0.793878,11.279936,2.799320,7.008163,2.187755,4.123129,2.723603,2.728246,2.761436,2.729932,3.153741
std,9.133301,8.105026,1.023933,1.106689,47068.888559,2.498887,3.659108,0.851883,7.782222,1.288978,6.125135,3.221699,3.567327,1.092756,1.101253,0.706245,0.711400,0.360742
min,18.000000,1.000000,1.000000,1.000000,10090.000000,0.000000,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,3.000000
25%,30.000000,2.000000,2.000000,1.000000,29110.000000,1.000000,12.000000,0.000000,6.000000,2.000000,3.000000,0.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000
50%,36.000000,7.000000,3.000000,2.000000,49190.000000,2.000000,14.000000,1.000000,10.000000,3.000000,5.000000,1.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
75%,43.000000,14.000000,4.000000,3.000000,83800.000000,4.000000,18.000000,1.000000,15.000000,3.000000,9.000000,3.000000,7.000000,4.000000,4.000000,3.000000,3.000000,3.000000
max,60.000000,29.000000,5.000000,5.000000,199990.000000,9.000000,25.000000,3.000000,40.000000,6.000000,40.000000,15.000000,17.000000,4.000000,4.000000,4.000000,4.000000,4.000000


In [5]:
# Display the data type for each variable
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4410 non-null   int64  
 1   Attrition                4410 non-null   object 
 2   BusinessTravel           4410 non-null   object 
 3   Department               4410 non-null   object 
 4   DistanceFromHome         4410 non-null   int64  
 5   Education                4410 non-null   int64  
 6   EducationField           4410 non-null   object 
 7   Gender                   4410 non-null   object 
 8   JobLevel                 4410 non-null   int64  
 9   JobRole                  4410 non-null   object 
 10  MaritalStatus            4410 non-null   object 
 11  MonthlyIncome            4410 non-null   int64  
 12  NumCompaniesWorked       4391 non-null   float64
 13  PercentSalaryHike        4410 non-null   int64  
 14  StockOptionLevel        

In [6]:
# DATA CLEANING
# Step 1: Handle missing values
# Replace missing numeric values with the median of the respective column.
# Median is more resilient to outliers than the mean.

# Look if the null values are greater than 0, if so, add them into a list
columns_with_na = df.columns[df.isnull().sum() > 0].tolist()

# Replacement
for col in columns_with_na:
    # Calculate median of the column
    median_value = df[col].median()
    # if there is a blank space, fill it with the median
    df.fillna({col: median_value}, inplace=True)

In [7]:
# Step 2: Convert categorical variables to numerical format using one-hot encoding
# drop_first=True avoids logical errors (multicollinearity)
df = pd.get_dummies(df, drop_first=True)

# Check the cleaned dataset structure
df.head(), df.isnull().sum().sum()  # Checking for any remaining missing values

(   Age  DistanceFromHome  Education  JobLevel  MonthlyIncome  \
 0   51                 6          2         1         131160   
 1   31                10          1         1          41890   
 2   32                17          4         4         193280   
 3   38                 2          5         3          83210   
 4   32                10          1         1          23420   
 
    NumCompaniesWorked  PercentSalaryHike  StockOptionLevel  TotalWorkingYears  \
 0                 1.0                 11                 0                1.0   
 1                 0.0                 23                 1                6.0   
 2                 1.0                 15                 3                5.0   
 3                 3.0                 11                 3               13.0   
 4                 4.0                 12                 2                9.0   
 
    TrainingTimesLastYear  ...  JobRole_Human Resources  \
 0                      6  ...                    False

In [8]:
# MODEL CONSTRUCTION - Determine relevant variables
# Separate the data into two groups: employees who left (Attrition = Yes) and those who stayed (Attrition = No)
data_left = df[df['Attrition_Yes'] == 1]
data_stayed = df[df['Attrition_Yes'] == 0]

# Define the output and input variables for each data set
# For employees who left
# Remove JobSatisfaction from the input variables since it is the output variable
X_left = data_left.drop(columns=["JobSatisfaction"])
# The output variable is JobSatisfaction
y_left = data_left["JobSatisfaction"]

# For employees who stayed
# Remove JobSatisfaction from the input variables since it is the output variable
X_stayed = data_stayed.drop(columns=["JobSatisfaction"])
# The output variable is JobSatisfaction
y_stayed = data_stayed["JobSatisfaction"]

In [19]:
# Initialize k fold cross-validation with 5 folds
# shuffle=True mixes the date before making the fold to guarantee a balanced sample
# random_state=24 is the seed, it determines reproducibility and should keep an integer constant value
kf = KFold(n_splits=5, shuffle=True, random_state=24)

# Function to perform k fold cross-validation and register feature importance
def calculate_feature_importances(X, y):
    # Initialize lists to store importances and errors
    fold_importances = []
    fold_errors = []
    training_errors = []
    # Loop through each fold
    for train_index, test_index in kf.split(X):
        # Split data into train and test sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        # Initialize a random forest of 100 trees and a constant value for the seed
        model = RandomForestRegressor(n_estimators=100, max_depth=9, random_state=24)
        # Train the model with the training set
        model.fit(X_train, y_train)
        # Predict using the test set of X values
        y_pred_test = model.predict(X_test)
        y_pred_train = model.predict(X_train)
        # Compute MSE between the predicted value and the real one and add it to a list
        fold_errors.append(mean_squared_error(y_test, y_pred_test))
        training_errors.append(mean_squared_error(y_train, y_pred_train))
        # Compute Gini impurity for each feature and add it to a list
        fold_importances.append(model.feature_importances_)
    # Average feature importances and errors across all folds
    avg_importances = np.mean(fold_importances, axis=0)
    avg_test_error = np.mean(fold_errors)
    avg_train_error = np.mean(training_errors)
    # Return the average importances and error
    return avg_importances, avg_test_error, avg_train_error, fold_errors, training_errors

# Calculate feature importances for each group
importances_left, error_left, avg_train_error1, test_errors1, train_errors1 = calculate_feature_importances(X_left, y_left)
importances_stayed, error_stayed, avg_train_error, test_errors, train_errors = calculate_feature_importances(X_stayed, y_stayed)
# Compute correlation
correlation_stayed = X_stayed.corrwith(y_stayed)
# Create dataframe
impact_analysis = pd.DataFrame({
    "Importance": importances_stayed,
    "Correlation": correlation_stayed
})
# Determine if the correlation is negative or positive
impact_analysis["Relationship"] = impact_analysis["Correlation"].apply(
    lambda x: "Proportional" if x > 0 else "Inversely Proportional"
)
# Sort by importance
impact_analysis = impact_analysis.sort_values(by="Importance", ascending=False)
# Store the feature importances in DataFrames for better visualization
feature_importances_left = pd.DataFrame({"Feature": X_left.columns, "Importance": importances_left}).sort_values(by="Importance", ascending=False)
# Display results
print("Top 10 most relevant features for job satisfaction")
print("\nFor employees who left")
print(feature_importances_left.head(10))
print("\nMean Squared Error for employees who left:", error_left)
print("\nFor employees who stayed:")
print(impact_analysis.head(10))
print("\nMean Squared Error for emplyees who stayed:", error_stayed)
print(f"Average Test Error (MSE): {error_stayed}")
print(f"Average Training Error (MSE): {avg_train_error}")
print(f"Test Errors by Fold: {test_errors}")
print(f"Training Errors by Fold: {train_errors}")

Top 10 most relevant features for job satisfaction

For employees who left
                    Feature  Importance
4             MonthlyIncome    0.119960
6         PercentSalaryHike    0.096250
1          DistanceFromHome    0.066207
8         TotalWorkingYears    0.064789
0                       Age    0.058377
10           YearsAtCompany    0.048926
5        NumCompaniesWorked    0.048185
11  YearsSinceLastPromotion    0.044524
12     YearsWithCurrManager    0.044113
14          WorkLifeBalance    0.039678

Mean Squared Error for employees who left: 0.27377253605816787

For employees who stayed:
                         Importance  Correlation            Relationship
MonthlyIncome              0.121801     0.001449            Proportional
DistanceFromHome           0.083307    -0.003997  Inversely Proportional
Age                        0.072922    -0.030337  Inversely Proportional
TotalWorkingYears          0.066103    -0.040719  Inversely Proportional
PercentSalaryHike          0.

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [ ]:
# Select from previous step the most logical relevant features into a list
top_5_features = X_stayed[["MonthlyIncome", "DistanceFromHome", "PercentSalaryHike",
                           "YearsSinceLastPromotion", "TrainingTimesLastYear"]]
# Declare the predictor variables
X_important_stayed = data_stayed[top_5_features.columns]
# Train the model with the selected features, 100 trees and the same seed
final_model = RandomForestRegressor(n_estimators=100, random_state=24)
final_model.fit(X_important_stayed, y_stayed)
# Calculate the magnitude of the adjustment needed for each variable in the model.
def calculate_adjustments(model, X, y, step=0.1):
    adjustments = []
    # Initial predictions from the model
    base_prediction = model.predict(X)
    for feature in X.columns:
        # Create a copy of X to modify only one variable at a time
        X_modified = X.copy()
        # Increment the variable by a small step
        X_modified[feature] += step
        increased_prediction = model.predict(X_modified)
        # Calculate the impact per unit of change
        impact_per_unit = (increased_prediction - base_prediction).mean() / step
        # Determine adjustment direction based on correlation
        correlation = X[feature].corr(y)
        direction = "Increase" if correlation > 0 else "Decrease"
        adjustments.append({
            "Feature": feature,
            "Impact per unit": impact_per_unit,
            "Suggested change": direction
        })
    # Convert to a DataFrame for analysis
    adjustment_df = pd.DataFrame(adjustments).sort_values(by="Impact per unit", ascending=False)
    return adjustment_df
# Calculate the necessary adjustments
adjustments = calculate_adjustments(final_model,top_5_features, y_stayed)
# Display the suggested adjustments
print("Adjustment recommendations:")
print(adjustments)

Adjustment recommendations:
                   Feature  Impact per unit Suggested change
0            MonthlyIncome         0.000014         Increase
4    TrainingTimesLastYear        -0.000243         Decrease
1         DistanceFromHome        -0.000622         Decrease
3  YearsSinceLastPromotion        -0.000885         Decrease
2        PercentSalaryHike        -0.001946         Increase


In [ ]:
# Function to try to compute recomendations
def suggest_optimized_adjustments(input_data, model, step=0.5, target_increase=0.5, max_satisfaction=4.0, max_iterations=500):
    # Initial prediction
    base_prediction = model.predict(pd.DataFrame(input_data, columns=top_5_features.columns))[0]
    target_prediction = min(base_prediction + target_increase, max_satisfaction)  # Cap at max satisfaction
    # Copy of input data for iterative adjustments
    adjusted_data = input_data.copy()
    # Compute correlations to determine postive or negative relationship
    correlations = {}
    for idx, feature in enumerate(top_5_features):
        correlations[feature] = data_stayed[feature].corr(data_stayed["JobSatisfaction"])
    # Optimization loop
    for iteration in range(max_iterations):
        # Initialize total adjustments per variable
        total_adjustments = []
        adjustments_applied = False
        # Iterate over each feature
        for idx, feature in enumerate(top_5_features):
            if feature == "MonthlyIncome":
                percent_salary_hike_index = top_5_features.columns.get_loc("PercentSalaryHike")
                if adjusted_data[0, percent_salary_hike_index] > 30 and all(
                    adjusted_data[0, i] <= 0 for i in range(len(top_5_features)) if top_5_features[i] != "MonthlyIncome"
                ):
                    # Adjust MonthlyIncome with a step of 100
                    adjusted_data[0, idx] += 100
                    new_prediction = model.predict(pd.DataFrame(adjusted_data, columns=top_5_features))[0]

                    if new_prediction >= target_prediction:
                        return {
                            "Adjusted Inputs": dict(zip(top_5_features, adjusted_data[0])),
                            "New Prediction": new_prediction,
                            "Iterations": iteration + 1
                        }
                    adjustments_applied = True
                    continue
            input_modified = adjusted_data.copy()
            # Respect the relationship
            relationship_direction = 1 if correlations[feature] > 0 else -1
            # Compute proposed value
            proposed_value = adjusted_data[0, idx] + step * relationship_direction
            # Enforce positivity
            if proposed_value <= 0:
              continue
            # Additional condition for PercentSalaryHike
            if feature == "PercentSalaryHike" and proposed_value > 30:
                continue
            # Predict with adjusted inputs
            input_modified[0, idx] = proposed_value
            new_prediction = model.predict(pd.DataFrame(input_modified, columns=top_5_features.columns))[0]
            # Calculate impact per unit
            impact_per_unit = (new_prediction - base_prediction) / step
            # Skip variables with negligible impact
            if abs(impact_per_unit) > 0:
                adjustment_step = step * relationship_direction
                total_adjustments.append((idx, adjustment_step, impact_per_unit))
                adjustments_applied = True
        # Apply adjustments proportionally to impact per unit
        for idx, adjustment_step, _ in total_adjustments:
            adjusted_data[0, idx] += adjustment_step
        # Predict with adjusted inputs
        new_prediction = model.predict(pd.DataFrame(adjusted_data, columns=top_5_features.columns))[0]
        # Check if target satisfaction is reached
        if new_prediction >= target_prediction:
            return {
                "Adjusted Inputs": dict(zip(top_5_features.columns, adjusted_data[0])),
                "New Prediction": new_prediction,
                "Iterations": iteration + 1
            }
        # Stop if no adjustments could be applied
        if not adjustments_applied:
            return {
                "Message": "No further optimization possible while maintaining positive values.",
                "Adjusted Inputs": dict(zip(top_5_features, adjusted_data[0])),
                "Final Prediction": new_prediction
            }
    # If max iterations are reached without achieving the target
    return {
        "Message": "Max iterations reached without achieving target increase.",
        "Adjusted Inputs": dict(zip(top_5_features.columns, adjusted_data[0])),
        "Final Prediction": new_prediction
    }
# Display the features along with their range and mean for bettter comprehension
print("Relevant features range for job satisfaction:")
for feature in top_5_features:
    min_val = data_stayed[feature].min()
    max_val = data_stayed[feature].max()
    mean_val = data_stayed[feature].mean()
    print(f"{feature} - Min: {min_val}, Max: {max_val}, Avg: {mean_val:.2f}")
# Function to ask for input values and predict satisfaction
def predict_satisfaction():
    print("\nEnter the following data to determine job satisfaction:")
    # Initialize a list for input data
    input_data = []
    # Store the recieved data in the list
    for feature in top_5_features:
        value = float(input(f"{feature}: "))
        input_data.append(value)
    # Convert to numpy array for data processing
    input_data = np.array([input_data])
    # Try the model with the input data
    satisfaction_prediction = final_model.predict(pd.DataFrame(input_data, columns=top_5_features.columns))[0]
    # Classify the result into a traffic light scale
    if satisfaction_prediction <= 2:
        traffic_light = "Red"
    elif 2 < satisfaction_prediction <= 3:
        traffic_light = "Yellow"
    else:
        traffic_light = "Green"
    # Display the results
    print(f"\nPredicted satisfaction: {satisfaction_prediction:.2f}")
    print(f"Traffic light scale: {traffic_light}")
    if traffic_light != "Green":
        print("\nOptimizing adjustments to improve satisfaction...")
        optimization_result = suggest_optimized_adjustments(input_data, final_model, step=0.5, target_increase=0.5, max_satisfaction=4.0)
        print("Optimization Results:")
        for key, value in optimization_result.items():
            print(f"{key}: {value}")
    else:
        print("\nSatisfaction is already at green level. No adjustments needed.")
# Call the function to predict
predict_satisfaction()

Relevant features range for job satisfaction:
MonthlyIncome - Min: 10510, Max: 199990, Avg: 65672.60
DistanceFromHome - Min: 1, Max: 29, Avg: 9.23
PercentSalaryHike - Min: 11, Max: 25, Avg: 15.16
YearsSinceLastPromotion - Min: 0, Max: 15, Avg: 2.23
TrainingTimesLastYear - Min: 0, Max: 6, Avg: 2.83

Enter the following data to determine job satisfaction:
MonthlyIncome: 50000
DistanceFromHome: 19
PercentSalaryHike: 5
YearsSinceLastPromotion: 5
TrainingTimesLastYear: 5

Predicted satisfaction: 2.72
Traffic light scale: Yellow

Optimizing adjustments to improve satisfaction...
Optimization Results:
Adjusted Inputs: {'MonthlyIncome': 50003.0, 'DistanceFromHome': 15.5, 'PercentSalaryHike': 8.0, 'YearsSinceLastPromotion': 1.5, 'TrainingTimesLastYear': 1.5}
New Prediction: 3.27
Iterations: 7
